# Binary Classifier: Waffles vs Pancakes

Run the code below to download the dataset `waffle_pancakes.zip`.

In [1]:
!gdown --id 1tJqtoF-5MNdC90rXYgtsXTs9o2odzEg5

Downloading...
From: https://drive.google.com/uc?id=1tJqtoF-5MNdC90rXYgtsXTs9o2odzEg5
To: /Users/chrismoroney/Documents/GitHub/waffles-vs-pancakes/waffle_pancakes.zip
13.0MB [00:00, 25.7MB/s]


In [12]:
import os
import zipfile

local_zip = './waffle_pancakes.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [13]:
# Directories with our training images
train_waf_dir = os.path.join('./waffle_pancakes/train/waffles')
train_pan_dir = os.path.join('./waffle_pancakes/train/pancakes')

# Directories with our test images
test_waf_dir = os.path.join('./waffle_pancakes/test/waffles')
test_pan_dir = os.path.join('./waffle_pancakes/test/pancakes')

In [14]:
train_waf_names = os.listdir(train_waf_dir)
print(train_waf_names[:10])
train_pan_names = os.listdir(train_pan_dir)
print(train_pan_names[:10])

test_waf_names = os.listdir(test_waf_dir)
test_pan_names = os.listdir(test_pan_dir)

['img-waf-153.png', 'img-waf-147.png', 'img-waf-190.png', 'img-waf-184.png', 'img-waf-437.png', 'img-waf-351.png', 'img-waf-345.png', 'img-waf-423.png', 'img-waf-379.png', 'img-waf-392.png']
['img-pan-59.png', 'img-pan-65.png', 'img-pan-71.png', 'img-pan-290.png', 'img-pan-284.png', 'img-pan-253.png', 'img-pan-247.png', 'img-pan-3.png', 'img-pan-327.png', 'img-pan-333.png']


In [15]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [16]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        './waffle_pancakes/train/',  # This is the source directory for training images
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary'
        )

validation_generator = validation_datagen.flow_from_directory(
        './waffle_pancakes/test/',  # This is the source directory for training images
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary'
        )

Found 947 images belonging to 2 classes.
Found 406 images belonging to 2 classes.


In [10]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=5,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8
      )

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")